# One-Way ANOVA

Autor: Luca Honegger, lh@sinnhaft.ch

Quelle: Dies ist eine Python-Portierung der UXtoolbox in R von Mohsen Rafiei, https://github.com/mohsen-rafiei/UXtoolbox


## 🧠 Was macht diese Funktion?

Die Funktion `anova_test()` führt eine **Einweg-ANOVA (One-Way ANOVA)** in Python durch.  
Sie prüft, ob sich **drei oder mehr unabhängige Gruppen** in einem quantitativen Merkmal (z. B. UX-Score) signifikant voneinander unterscheiden.

### 🔍 Was wird berechnet?

* Ob es **signifikante Mittelwertunterschiede** zwischen den Gruppen gibt
* Ob die Voraussetzungen für ANOVA erfüllt sind:
  * **Normalverteilung** (Shapiro-Wilk-Test über `scipy`)
  * **Varianzhomogenität** (Levene-Test über `scipy`)
* Welche ANOVA verwendet wird:
  * **Standard-ANOVA** (mit `statsmodels`, falls Varianzen gleich)
  * **Welch-ANOVA** (mit `pingouin`, falls Varianzen ungleich)
* Die **Effektgrösse η² (Eta-Quadrat)** – misst, wie viel der Gesamtvarianz durch Gruppenzugehörigkeit erklärt wird
* Optional: **Post-hoc-Test (Tukey HSD)** mit `statsmodels`, um zu prüfen, **welche Gruppen sich konkret unterscheiden**

Das ist besonders nützlich, wenn du z. B. verschiedene UX-Designs vergleichst und wissen willst, ob diese einen signifikanten Unterschied in der Nutzerbewertung bewirken – und falls ja, **wo genau**.

## 🧪 Beispiel aus dem Business-Alltag

Stell dir vor, du testest drei Layout-Varianten einer App:

* **Gruppe A** – klassisches Layout  
* **Gruppe B** – modernes Layout  
* **Gruppe C** – reduziertes Layout 

Jede Testperson bewertet die Nutzererfahrung auf einer Skala von 0 bis 100 (z.B. System Usability Scale, SUS). Deine Testdaten sehen z. B. so aus:

```python
df = pd.DataFrame({
    'Group': ['A'] * 5 + ['B'] * 5 + ['C'] * 5,
    'Score': [
        58, 59, 60, 60, 61,   # Gruppe A
        61, 63, 62, 63, 64,   # Gruppe B
        60, 61, 60, 61, 62    # Gruppe C
    ]
})
```

Du erhältst:

- Ob es statistisch signifikante Unterschiede zwischen den Gruppen gibt
- Welche Gruppen sich signifikant voneinander unterscheiden
- Die Stärke des Effekts (η²)
- Und ob die statistischen Annahmen erfüllt sind

Ideal für UX-Research, A/B/C-Tests, Prototypenvergleich oder auch qualitative Nutzerfeedback-Auswertung.

## Funktion `anova_test`

In [88]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def anova_test(data, group_col, value_col):
    results = {}

    # Sicherstellen, dass Gruppierung als Kategorie vorliegt
    data[group_col] = data[group_col].astype("category")

    # Normalverteilungstest
    shapiro_test = stats.shapiro(data[value_col])
    results["Normality (Shapiro-Wilk)"] = {
        "W": shapiro_test.statistic,
        "p": shapiro_test.pvalue
    }

    # Levene-Test (Varianzgleichheit)
    groups = [
        group[value_col].values
        for name, group in data.groupby(group_col, observed=False)  # <- Fix für Warning
    ]
    levene_test = stats.levene(*groups)
    results["Homogeneity (Levene)"] = {
        "W": levene_test.statistic,
        "p": levene_test.pvalue
    }

    # Welch-ANOVA (vereinfachte Version)
    if levene_test.pvalue < 0.05:
        f_stat, p_val = stats.f_oneway(*groups)
        results["ANOVA Type"] = "Welch's ANOVA (vereinfachte Version)"
        results["ANOVA Summary"] = {
            "F": f_stat,
            "p": p_val
        }
        results["Effect Size (η²)"] = "Nicht berechnet (für Welch-ANOVA nicht direkt anwendbar)"
        results["Post-Hoc (Tukey)"] = "Nicht verfügbar bei Welch-ANOVA"

    else:
        # Standard-ANOVA
        model = ols(f'{value_col} ~ C({group_col})', data=data).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        results["ANOVA Type"] = "Standard ANOVA"
        results["ANOVA Summary"] = anova_table.reset_index().to_dict(orient="records")

        # Effektgrösse η² (Fix für .iloc statt [0])
        ss_between = anova_table["sum_sq"].iloc[0]
        ss_total = anova_table["sum_sq"].sum()
        eta_squared = ss_between / ss_total
        results["Effect Size (η²)"] = eta_squared

        # Post-hoc-Test
        tukey = pairwise_tukeyhsd(endog=data[value_col],
                                  groups=data[group_col],
                                  alpha=0.05)
        results["Post-Hoc (Tukey)"] = pd.DataFrame(
            data=tukey.summary().data[1:],
            columns=tukey.summary().data[0]
        ).to_dict(orient="records")

    return results

## Anwendungsbeispiel

In [90]:
# Beispieldaten
import pandas as pd

df = pd.DataFrame({
    'Group': ['A'] * 5 + ['B'] * 5 + ['C'] * 5,
    'Score': [
        58, 59, 60, 60, 61,   # Gruppe A
        61, 63, 62, 63, 64,   # Gruppe B
        60, 61, 60, 61, 62    # Gruppe C
    ]
})

In [92]:
# ANOVA durchführen
result = anova_test(df, group_col="Group", value_col="Score")

# Ergebnisse lesbar ausgeben
print(f"ANOVA-Typ: {result['ANOVA Type']}")
if isinstance(result['Effect Size (η²)'], float):
    print(f"Effektgrösse η²: {result['Effect Size (η²)']:.3f}")
else:
    print(f"Effektgrösse: {result['Effect Size (η²)']}")

print("\nErgebnis der ANOVA:")
for row in result['ANOVA Summary']:
    print(f"{row}")

# 👉 ANOVA-p-Wert explizit extrahieren (erste Zeile enthält den Gruppenvergleich)
anova_p = result["ANOVA Summary"][0].get("PR(>F)")
if anova_p is not None:
    print(f"\np-Wert der ANOVA: {anova_p:.3f}")

print("\nVoraussetzungen (Annahmen-Prüfung):")
print(f"Shapiro-Wilk (Normalverteilung): p = {result['Normality (Shapiro-Wilk)']['p']:.3f}")
print(f"Levene-Test (Varianzgleichheit): p = {result['Homogeneity (Levene)']['p']:.3f}")

# Optional: Post-hoc-Ergebnisse
if isinstance(result['Post-Hoc (Tukey)'], list):
    print("\nPost-hoc Tukey-Test (nur bei Standard-ANOVA):")
    for row in result['Post-Hoc (Tukey)']:
        print(f"{row['group1']} vs {row['group2']}: p = {row['p-adj']:.3f}, sig. = {row['reject']}")
else:
    print("\nPost-hoc-Test: " + result['Post-Hoc (Tukey)'])


ANOVA-Typ: Standard ANOVA
Effektgrösse η²: 0.633

Ergebnis der ANOVA:
{'index': 'C(Group)', 'sum_sq': 22.800000000000228, 'df': 2.0, 'F': 10.363636363636468, 'PR(>F)': 0.0024301248285321445}
{'index': 'Residual', 'sum_sq': 13.2, 'df': 12.0, 'F': nan, 'PR(>F)': nan}

p-Wert der ANOVA: 0.002

Voraussetzungen (Annahmen-Prüfung):
Shapiro-Wilk (Normalverteilung): p = 0.785
Levene-Test (Varianzgleichheit): p = 0.890

Post-hoc Tukey-Test (nur bei Standard-ANOVA):
A vs B: p = 0.002, sig. = True
A vs C: p = 0.208, sig. = False
B vs C: p = 0.046, sig. = True


### 📊 Erklärung und Interpretation

#### 🧠 Zusammenfassung

Die beispielhafte durchgeführte **Standard-ANOVA** zeigt, dass es signifikante Unterschiede zwischen den drei Gruppen gibt.

- **ANOVA-Typ:** Standard ANOVA  
- **Effektgrösse η²:** 0.633  
  → Das bedeutet, dass **63.3 % der Gesamtvarianz** durch die Gruppenzugehörigkeit erklärt werden kann – ein **sehr starker Effekt**.


#### 🔍 ANOVA-Ergebnis

| Effekt | Summe der Quadrate (SS) | df | F-Wert | p-Wert |
|--------|--------------------------|----|--------|--------|
| Gruppenunterschied (C(Group)) | 22.80 | 2 | 10.36 | **0.002** |
| Residual (Fehlervarianz) | 13.20 | 12 | — | — |

➡️ **p = 0.002** zeigt, dass der Unterschied **statistisch signifikant** ist (unter dem üblichen Schwellenwert von 0.05).


#### ✅ Voraussetzungen geprüft

- **Normalverteilung:** Shapiro-Wilk p = 0.785 → ✅ keine Abweichung von Normalverteilung  
- **Varianzgleichheit:** Levene-Test p = 0.890 → ✅ Varianzen sind homogen

Die Voraussetzungen für eine Standard-ANOVA sind somit **erfüllt**.


#### 🔎 Post-hoc-Vergleiche (Tukey HSD)

| Vergleich | p-Wert | Signifikant? |
|-----------|--------|--------------|
| A vs B    | 0.002  | ✅ Ja         |
| A vs C    | 0.208  | ❌ Nein       |
| B vs C    | 0.046  | ✅ Ja         |

➡️ **Gruppe B unterscheidet sich signifikant** von Gruppe A **und** von Gruppe C.  
➡️ **Gruppe A und C unterscheiden sich nicht signifikant**.


#### 📌 Fazit

Die Daten zeigen **deutliche Unterschiede zwischen den Gruppen**, insbesondere Gruppe B hebt sich von den anderen beiden ab.  
Dank hoher Effektgrösse (η² > 0.6) ist der Unterschied auch **praktisch bedeutsam** – nicht nur statistisch.